In [160]:
import glob
import pdfplumber
import re
import pandas as pd
import datetime as dt
import pymysql
from pathlib import Path
import os
import re

from datetime import date
from dateutil.relativedelta import relativedelta


In [271]:
#Função que muda a exibição da data de: 02/2022 para: 2022-02-01
def ConverterData(txData):
    return dt.datetime.strptime('01/' + txData, "%d/%m/%Y").date()

def ConectaBD():
    return pymysql.connect(
    host='localhost',
    user='master',
    password='123456',
    database='unicacontabil',
    cursorclass = pymysql.cursors.DictCursor
)


In [272]:
# Processar PGDAS e gravar dados no banco de dados
path = './Arquivamento/*/*/PGDAS.pdf'
for file in glob.iglob(path ):
    pdf = pdfplumber.open(file)

    pdf_text = ''
    for pg in pdf.pages:
        pdf_text += pg.extract_text()
    pdf.close()

    # CNPJ Estabelecimento
    cnpj = re.findall(r'[0-9]{2}\.[0-9]{3}\.[0-9]{3}\/[0-9]{4}-[0-9]{2}', pdf_text)

    # Data de Referencia da Apuração
    tpDoc = pdf_text[0:15].split()[0]
    if tpDoc == 'Extrato': #PGDAS-R Extrato
        dtref = re.findall(r'Período de Apuração \(PA\): ([0-9]{2}\/[0-9]{4})', pdf_text)
    else:
        dtref = re.findall(r'Período de Apuração: [0-9]{2}\/([0-9]{2}\/[0-9]{4})', pdf_text)

    # Receita Bruta do Mês ([0]VL_MI, [1]VL_ME, [2]VL_TOTAL)
    rec = re.findall(r'Receita Bruta do PA \(RPA\) - Competência (.*)', pdf_text)
    receita = rec[0].replace('.','').replace(',', '.').split(' ')
    # tx

    inicio = pdf_text.find('2.2.1) Mercado Interno') + len('2.2.1) Mercado Interno')
    fim = pdf_text.find('2.2.2) Mercado Externo')
    tx = pdf_text[inicio:fim].replace('\n', ' ').replace('.','').replace(',','.').replace(' ','')
    mi = re.findall(r'([0-9]{2}/[0-9]{4})([0-9]+\.[0-9]{2})', tx)
    
    #Add tupla com os dados do mes atual na list mi
    mi.append((dtref[0], receita[0]))

    inicio = pdf_text.find('2.2.2) Mercado Externo') + len('2.2.2) Mercado Externo')
    fim = pdf_text.find('2.3) Folha de Salários Anteriores (R$)')
    tx = pdf_text[inicio:fim].replace('\n', ' ').replace('.','').replace(',','.').replace(' ','')
    me = re.findall(r'([0-9]{2}/[0-9]{4})([0-9]+\.[0-9]{2})', tx)

    #Add tupla com os dados do mes atual na list me
    me.append((dtref[0], receita[1]))


    # Dataframes
    dfMI = pd.DataFrame(mi, columns=['DT_REF', 'VL_MI'])
    dfME = pd.DataFrame(me, columns=['DT_REF', 'VL_ME'])

    dfMI.set_index('DT_REF', inplace=True)
    dfME.set_index('DT_REF', inplace=True)

    df = dfMI.join(dfME)

    df = df.apply(pd.to_numeric, errors='coerce')
    df['VL_TOTAL'] = df['VL_MI'] + df['VL_ME']

    # Gravar cada linha do df no banco de dados
    for i, r in df.iterrows():
        con = pymysql.connect(
            host='localhost',
            user='master',
            password='123456',
            database='unicacontabil'
        )

        sql = "INSERT INTO histreceitasbrutas (CNPJ, DT_COMPETENCIA, VL_MI, VL_ME) VALUES ('{}', '{}', {}, {})".format(cnpj[0], ConverterData(i), r[0], r[1])

        try:
            c = con.cursor()
            c.execute(sql)
            con.commit()
        except Exception as e:
            pass
            # print(e.__doc__)
        finally:
            c.close()
            if con.open:
                con.close()
                # print('Fechado')




In [273]:
# Ler Excel com as empresas que devem ser calculadas
path_download = os.path.join(Path.home(), "Downloads")

# arq = 'C:/Users/Henrique/Downloads/ALÍQUOTAS MENSAIS.xlsx'
arq = os.path.join(path_download, 'ALÍQUOTAS MENSAIS.xlsx')

df = pd.read_excel(arq, header=2)
# df2 = df.query("OBSERVAÇÕES != 'NÃO ENVIAR POR E-MAIL'").iloc[:, 0:29]
df.set_index('CNPJ', inplace=True)

df2 = df
df2["ANEXO SIMPLES"].replace('FATOR R' , 'ANEXO 3' , regex=True, inplace=True)


In [274]:
def formataCNPJ(cnpj):
    cnpj = str("{:0>14}".format(int(cnpj)))
    return cnpj[0:2] + '.' + cnpj[2:5] + '.' + cnpj[5:8] + '/' + cnpj[8:12] + '-' + cnpj[12:14]

# for i, r in df2.iterrows():
#     print(formataCNPJ(i))

In [276]:
log = open("log.txt","w") 


# Calculo
for i, r in df2.iterrows():
    log.write(formataCNPJ(i) +'\n') 

    dtMin = '2021-07-01'
    dtMax = '2022-06-01'
    sql = """SELECT min(DT_COMPETENCIA) DT_MIN, max(DT_COMPETENCIA) DT_MAX, SUM(VL_TOTAL) FATURAMENTO, COUNT(CNPJ) QT_MESES FROM histreceitasbrutas 
    WHERE CNPJ = '{}' and 
    DT_COMPETENCIA BETWEEN '{}' AND '{}'""" .format(formataCNPJ(i), dtMin, dtMax)

    con = ConectaBD()

    try:
        c = con.cursor()
        c.execute(sql)
        # rows = c.fetchall()
        row = c.fetchone()
    except Exception as e:
        pass
        # print(e.__doc__)
    finally:
        c.close()
        if con.open:
            con.close()
            # print('Fechado')

    if (row['FATURAMENTO'] is not None and row['FATURAMENTO'] < 4800000 and row['FATURAMENTO'] > 0):
        fatBase = float(row['FATURAMENTO']) / float(row['QT_MESES']) * 12

        con = ConectaBD()

        sql = "SELECT * FROM aliquotas a WHERE a.TP_DOC = '{}' AND {} BETWEEN a.DE AND a.ATE" .format( r['ANEXO SIMPLES'], fatBase)

        try:
            c = con.cursor()
            c.execute(sql)
            # rows = c.fetchall()
            x = c.fetchone()
        except Exception as e:
            pass
            # print(e.__doc__)
        finally:
            c.close()
            if con.open:
                con.close()
                # print('Fechado')

        total = (fatBase * x['ALIQUOTA'] - x['DESCONTO']) / fatBase * 100

        iss_icms = total * x['ISS_ICMS'] 

        # Gravar no BD a apuração
        con = pymysql.connect(
            host='localhost',
            user='master',
            password='123456',
            database='unicacontabil'
        )

        sql = """INSERT INTO apuração 
(DT_REFERENCIA, CNPJ, ANEXO, FATURAMENTO, DT_MIN, DT_MAX, QT_MESES, RBT12, EFETIVA, ISS_ICMS) 
VALUES ('{}', '{}', '{}', {}, '{}', '{}', {}, {}, {}, {})""" .format(dtMax, formataCNPJ(i), r['ANEXO SIMPLES'], row['FATURAMENTO'], row['DT_MIN'], row['DT_MAX'], row['QT_MESES'], fatBase, total, iss_icms)

        try:
            c = con.cursor()
            c.execute(sql)
            con.commit()
        except Exception as e:
            pass
            # print(e.__doc__)
        finally:
            c.close()
            if con.open:
                con.close()
                # print('Fechado')

log.close() 


In [260]:
log.close()

In [129]:
# import packages
old_date = date(2022, 6, 1)  
print('date : ' + str(old_date))
new_date = old_date + relativedelta(months=-11)
print('new date is : '+str(new_date))

date : 2022-06-01
new date is : 2021-07-01
